# PyTorch Tutorial: Genetic Algorithms and Neuroevolution

Genetic Algorithms (GAs) are optimization algorithms inspired by natural selection. **Neuroevolution** applies GAs to evolve neural network weights (or architectures) without using gradient descent.

## Learning Objectives
- Understand genetic algorithm fundamentals (selection, crossover, mutation)
- Implement a basic GA for function optimization
- Evolve neural network weights to solve control tasks
- Learn about NEAT (NeuroEvolution of Augmenting Topologies)
- Compare GAs with gradient-based methods and RL

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Callable
import copy

np.random.seed(42)
torch.manual_seed(42)

## 1. Genetic Algorithm Fundamentals

A genetic algorithm works by:
1. **Initialize** a population of candidate solutions (genomes)
2. **Evaluate** each individual's fitness
3. **Select** the fittest individuals
4. **Crossover** pairs to create offspring
5. **Mutate** offspring to introduce variation
6. **Repeat** until convergence

In [ ]:
class Individual:
    """Represents a single individual in the population."""
    
    def __init__(self, genome: np.ndarray):
        self.genome = genome
        self.fitness = None
    
    def __repr__(self):
        return f"Individual(fitness={self.fitness:.4f})" if self.fitness else "Individual(unevaluated)"


class GeneticAlgorithm:
    """Basic Genetic Algorithm implementation."""
    
    def __init__(
        self,
        genome_size: int,
        population_size: int = 50,
        mutation_rate: float = 0.1,
        mutation_strength: float = 0.5,
        elite_ratio: float = 0.1,
    ):
        self.genome_size = genome_size
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.mutation_strength = mutation_strength
        self.elite_count = max(1, int(population_size * elite_ratio))
        
        # Initialize random population
        self.population = [
            Individual(np.random.randn(genome_size))
            for _ in range(population_size)
        ]
    
    def evaluate_population(self, fitness_fn: Callable):
        """Evaluate fitness of all individuals."""
        for individual in self.population:
            individual.fitness = fitness_fn(individual.genome)
    
    def tournament_selection(self, tournament_size: int = 3) -> Individual:
        """Select individual via tournament selection."""
        tournament = np.random.choice(self.population, size=tournament_size, replace=False)
        return max(tournament, key=lambda x: x.fitness)
    
    def crossover(self, parent1: Individual, parent2: Individual) -> Individual:
        """Single-point crossover between two parents."""
        crossover_point = np.random.randint(1, self.genome_size)
        child_genome = np.concatenate([
            parent1.genome[:crossover_point],
            parent2.genome[crossover_point:]
        ])
        return Individual(child_genome)
    
    def mutate(self, individual: Individual) -> Individual:
        """Apply Gaussian mutation to genome."""
        mask = np.random.random(self.genome_size) < self.mutation_rate
        mutation = np.random.randn(self.genome_size) * self.mutation_strength
        new_genome = individual.genome + mask * mutation
        return Individual(new_genome)
    
    def evolve_generation(self, fitness_fn: Callable):
        """Create next generation through selection, crossover, and mutation."""
        # Evaluate current population
        self.evaluate_population(fitness_fn)
        
        # Sort by fitness (descending)
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        
        # Keep elite individuals
        new_population = [copy.deepcopy(ind) for ind in self.population[:self.elite_count]]
        
        # Fill rest with offspring
        while len(new_population) < self.population_size:
            parent1 = self.tournament_selection()
            parent2 = self.tournament_selection()
            child = self.crossover(parent1, parent2)
            child = self.mutate(child)
            new_population.append(child)
        
        self.population = new_population
    
    def get_best(self) -> Individual:
        """Return the best individual in the population."""
        return max(self.population, key=lambda x: x.fitness if x.fitness else float('-inf'))

### Example: Optimizing a Simple Function

Let's use GA to find the minimum of the Rastrigin function (a challenging multi-modal function).

In [ ]:
def rastrigin(x: np.ndarray) -> float:
    """Rastrigin function - global minimum at x=0."""
    A = 10
    n = len(x)
    # Negate because GA maximizes fitness
    return -(A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x)))

# Run GA
ga = GeneticAlgorithm(
    genome_size=10,
    population_size=100,
    mutation_rate=0.2,
    mutation_strength=0.3,
)

history = []
for generation in range(100):
    ga.evolve_generation(rastrigin)
    best = ga.get_best()
    history.append(-best.fitness)  # Convert back to actual function value
    
    if generation % 20 == 0:
        print(f"Gen {generation}: Best fitness = {-best.fitness:.4f}")

# Plot convergence
plt.figure(figsize=(10, 4))
plt.plot(history)
plt.xlabel('Generation')
plt.ylabel('Best Rastrigin Value')
plt.title('GA Optimization of Rastrigin Function')
plt.axhline(y=0, color='r', linestyle='--', label='Global minimum')
plt.legend()
plt.show()

print(f"\nFinal solution: {ga.get_best().genome[:5]}...")  # Show first 5 values
print(f"Final fitness: {-ga.get_best().fitness:.6f} (optimal: 0)")

## 2. Selection Strategies

Different selection methods have different properties:

In [ ]:
def roulette_wheel_selection(population: List[Individual]) -> Individual:
    """
    Fitness-proportionate selection.
    Higher fitness = higher probability of selection.
    """
    fitness_values = np.array([ind.fitness for ind in population])
    # Shift to positive if needed
    min_fitness = fitness_values.min()
    if min_fitness < 0:
        fitness_values = fitness_values - min_fitness + 1e-6
    
    probabilities = fitness_values / fitness_values.sum()
    return np.random.choice(population, p=probabilities)


def rank_selection(population: List[Individual]) -> Individual:
    """
    Rank-based selection.
    Selection probability based on rank, not raw fitness.
    Helps prevent premature convergence.
    """
    sorted_pop = sorted(population, key=lambda x: x.fitness)
    ranks = np.arange(1, len(population) + 1)
    probabilities = ranks / ranks.sum()
    return np.random.choice(sorted_pop, p=probabilities)


def truncation_selection(population: List[Individual], top_ratio: float = 0.5) -> Individual:
    """
    Select only from top performers.
    Simple but can lead to loss of diversity.
    """
    sorted_pop = sorted(population, key=lambda x: x.fitness, reverse=True)
    cutoff = max(1, int(len(population) * top_ratio))
    return np.random.choice(sorted_pop[:cutoff])


# Compare selection pressures
print("Selection Strategy Comparison:")
print("-" * 50)
print("| Strategy    | Pressure | Diversity | Speed |")
print("-" * 50)
print("| Tournament  | Medium   | Good      | Fast  |")
print("| Roulette    | Variable | Medium    | Fast  |")
print("| Rank        | Medium   | Good      | Fast  |")
print("| Truncation  | High     | Low       | Fast  |")
print("-" * 50)

## 3. Neuroevolution: Evolving Neural Networks

Now let's evolve neural network weights to solve a control task!

In [ ]:
class EvolvableNetwork(nn.Module):
    """A neural network whose weights can be set from a flat genome."""
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh(),
        )
        self.genome_size = self._count_parameters()
    
    def _count_parameters(self) -> int:
        return sum(p.numel() for p in self.parameters())
    
    def set_weights_from_genome(self, genome: np.ndarray):
        """Set network weights from a flat numpy array."""
        idx = 0
        with torch.no_grad():
            for param in self.parameters():
                param_size = param.numel()
                param_data = genome[idx:idx + param_size]
                param.copy_(torch.tensor(param_data, dtype=torch.float32).view(param.shape))
                idx += param_size
    
    def get_genome(self) -> np.ndarray:
        """Extract weights as a flat numpy array."""
        return np.concatenate([p.data.cpu().numpy().flatten() for p in self.parameters()])
    
    def forward(self, x):
        return self.network(x)

# Example network
net = EvolvableNetwork(input_size=4, hidden_size=8, output_size=2)
print(f"Network has {net.genome_size} parameters (genes)")

### Simple Control Task: Balance a Pole

We'll simulate a simple cart-pole-like environment.

In [ ]:
class SimplePoleEnvironment:
    """
    Simplified pole balancing task.
    State: [position, velocity, angle, angular_velocity]
    Action: force applied to cart [-1, 1]
    """
    
    def __init__(self):
        self.reset()
        
        # Physics constants
        self.gravity = 9.8
        self.cart_mass = 1.0
        self.pole_mass = 0.1
        self.pole_length = 0.5
        self.dt = 0.02
    
    def reset(self) -> np.ndarray:
        # Small random initial state
        self.state = np.random.uniform(-0.05, 0.05, size=4)
        return self.state.copy()
    
    def step(self, action: float) -> Tuple[np.ndarray, float, bool]:
        x, x_dot, theta, theta_dot = self.state
        force = np.clip(action, -1, 1) * 10  # Scale action
        
        # Simplified physics
        cos_theta = np.cos(theta)
        sin_theta = np.sin(theta)
        
        total_mass = self.cart_mass + self.pole_mass
        pole_mass_length = self.pole_mass * self.pole_length
        
        # Angular acceleration
        temp = (force + pole_mass_length * theta_dot**2 * sin_theta) / total_mass
        theta_acc = (self.gravity * sin_theta - cos_theta * temp) / (
            self.pole_length * (4/3 - self.pole_mass * cos_theta**2 / total_mass)
        )
        
        # Linear acceleration
        x_acc = temp - pole_mass_length * theta_acc * cos_theta / total_mass
        
        # Update state
        x += self.dt * x_dot
        x_dot += self.dt * x_acc
        theta += self.dt * theta_dot
        theta_dot += self.dt * theta_acc
        
        self.state = np.array([x, x_dot, theta, theta_dot])
        
        # Check termination
        done = abs(x) > 2.4 or abs(theta) > 0.21  # ~12 degrees
        reward = 1.0 if not done else 0.0
        
        return self.state.copy(), reward, done


def evaluate_network(network: EvolvableNetwork, env: SimplePoleEnvironment, max_steps: int = 500) -> float:
    """Evaluate network on the pole balancing task."""
    state = env.reset()
    total_reward = 0
    
    for _ in range(max_steps):
        # Get action from network
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
            action = network(state_tensor).item()
        
        state, reward, done = env.step(action)
        total_reward += reward
        
        if done:
            break
    
    return total_reward

### Neuroevolution Training Loop

In [ ]:
class Neuroevolution:
    """Evolve neural network weights using genetic algorithms."""
    
    def __init__(
        self,
        network_template: EvolvableNetwork,
        population_size: int = 50,
        mutation_rate: float = 0.1,
        mutation_strength: float = 0.2,
        elite_ratio: float = 0.1,
    ):
        self.network_template = network_template
        self.genome_size = network_template.genome_size
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.mutation_strength = mutation_strength
        self.elite_count = max(1, int(population_size * elite_ratio))
        
        # Initialize population
        self.population = [
            Individual(np.random.randn(self.genome_size) * 0.5)
            for _ in range(population_size)
        ]
    
    def evaluate_individual(self, genome: np.ndarray, env: SimplePoleEnvironment) -> float:
        """Evaluate a single genome."""
        network = copy.deepcopy(self.network_template)
        network.set_weights_from_genome(genome)
        
        # Average over multiple runs for stability
        total_fitness = 0
        n_runs = 3
        for _ in range(n_runs):
            total_fitness += evaluate_network(network, env)
        
        return total_fitness / n_runs
    
    def evolve_generation(self, env: SimplePoleEnvironment):
        """Evolve one generation."""
        # Evaluate fitness
        for individual in self.population:
            individual.fitness = self.evaluate_individual(individual.genome, env)
        
        # Sort by fitness
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        
        # Keep elites
        new_population = [copy.deepcopy(ind) for ind in self.population[:self.elite_count]]
        
        # Create offspring
        while len(new_population) < self.population_size:
            # Tournament selection
            tournament = np.random.choice(self.population, size=3, replace=False)
            parent1 = max(tournament, key=lambda x: x.fitness)
            tournament = np.random.choice(self.population, size=3, replace=False)
            parent2 = max(tournament, key=lambda x: x.fitness)
            
            # Crossover
            crossover_point = np.random.randint(1, self.genome_size)
            child_genome = np.concatenate([
                parent1.genome[:crossover_point],
                parent2.genome[crossover_point:]
            ])
            
            # Mutation
            mask = np.random.random(self.genome_size) < self.mutation_rate
            mutation = np.random.randn(self.genome_size) * self.mutation_strength
            child_genome = child_genome + mask * mutation
            
            new_population.append(Individual(child_genome))
        
        self.population = new_population
    
    def get_best_network(self) -> EvolvableNetwork:
        """Return the best network in the population."""
        best = max(self.population, key=lambda x: x.fitness if x.fitness else float('-inf'))
        network = copy.deepcopy(self.network_template)
        network.set_weights_from_genome(best.genome)
        return network

In [ ]:
# Create network template and environment
network_template = EvolvableNetwork(input_size=4, hidden_size=8, output_size=1)
env = SimplePoleEnvironment()

# Initialize neuroevolution
neuro = Neuroevolution(
    network_template=network_template,
    population_size=50,
    mutation_rate=0.1,
    mutation_strength=0.2,
)

print(f"Evolving networks with {network_template.genome_size} parameters...\n")

# Evolution loop
best_fitness_history = []
avg_fitness_history = []

for generation in range(50):
    neuro.evolve_generation(env)
    
    # Track statistics
    fitness_values = [ind.fitness for ind in neuro.population]
    best_fitness = max(fitness_values)
    avg_fitness = np.mean(fitness_values)
    
    best_fitness_history.append(best_fitness)
    avg_fitness_history.append(avg_fitness)
    
    if generation % 10 == 0:
        print(f"Gen {generation}: Best={best_fitness:.1f}, Avg={avg_fitness:.1f}")

# Plot results
plt.figure(figsize=(10, 4))
plt.plot(best_fitness_history, label='Best Fitness')
plt.plot(avg_fitness_history, label='Average Fitness', alpha=0.7)
plt.axhline(y=500, color='r', linestyle='--', label='Max possible (500)')
plt.xlabel('Generation')
plt.ylabel('Fitness (steps balanced)')
plt.title('Neuroevolution: Pole Balancing')
plt.legend()
plt.show()

print(f"\nFinal best fitness: {best_fitness_history[-1]:.1f}")

## 4. NEAT: NeuroEvolution of Augmenting Topologies

NEAT evolves both weights AND network topology (adding nodes and connections over time).

### Key NEAT Concepts:

1. **Historical Markings**: Track when genes were created to align genomes during crossover
2. **Speciation**: Group similar networks to protect innovation
3. **Complexification**: Start simple, add complexity only when needed

In [ ]:
# NEAT is complex to implement from scratch. Here's a simplified illustration:

class SimplifiedNEATGene:
    """Represents a connection gene in NEAT."""
    global_innovation = 0
    
    def __init__(self, in_node: int, out_node: int, weight: float, enabled: bool = True):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight
        self.enabled = enabled
        self.innovation = SimplifiedNEATGene.global_innovation
        SimplifiedNEATGene.global_innovation += 1
    
    def __repr__(self):
        status = "ON" if self.enabled else "OFF"
        return f"Gene({self.in_node}->{self.out_node}, w={self.weight:.2f}, {status}, inn={self.innovation})"


# Example genome
print("NEAT Genome Example:")
print("-" * 50)

genome = [
    SimplifiedNEATGene(1, 4, 0.5),   # Input 1 -> Hidden 4
    SimplifiedNEATGene(2, 4, -0.3),  # Input 2 -> Hidden 4
    SimplifiedNEATGene(4, 5, 0.8),   # Hidden 4 -> Output 5
    SimplifiedNEATGene(1, 5, 0.2),   # Direct connection: Input 1 -> Output 5
]

for gene in genome:
    print(gene)

print("\n" + "-" * 50)
print("NEAT allows the network to grow over generations!")
print("New nodes and connections can be added via mutation.")

## 5. Evolution Strategies (ES) - Modern Alternative

Evolution Strategies are a related approach that's simpler and often works better for neuroevolution.

In [ ]:
class EvolutionStrategy:
    """
    Simple Evolution Strategy (1+1)-ES or (mu, lambda)-ES.
    No crossover, just mutation and selection.
    """
    
    def __init__(
        self,
        genome_size: int,
        population_size: int = 50,
        sigma: float = 0.1,
        learning_rate: float = 0.01,
    ):
        self.genome_size = genome_size
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate
        
        # Initialize mean of distribution
        self.theta = np.zeros(genome_size)
    
    def sample_population(self) -> List[np.ndarray]:
        """Sample population around current mean."""
        noise = [np.random.randn(self.genome_size) for _ in range(self.population_size)]
        population = [self.theta + self.sigma * n for n in noise]
        return population, noise
    
    def update(self, fitness_values: np.ndarray, noise: List[np.ndarray]):
        """Update mean using fitness-weighted noise."""
        # Normalize fitness
        fitness_values = np.array(fitness_values)
        fitness_values = (fitness_values - fitness_values.mean()) / (fitness_values.std() + 1e-8)
        
        # Weighted sum of noise vectors
        gradient = np.zeros(self.genome_size)
        for f, n in zip(fitness_values, noise):
            gradient += f * n
        gradient /= self.population_size * self.sigma
        
        # Update
        self.theta += self.learning_rate * gradient

print("Evolution Strategies (ES):")
print("-" * 50)
print("1. Sample population: theta + sigma * noise")
print("2. Evaluate fitness of each sample")
print("3. Update theta toward high-fitness samples")
print("4. Repeat")
print("\nAdvantages over GA:")
print("- Simpler (no crossover)")
print("- Highly parallelizable")
print("- Works well for high-dimensional problems")

## 6. Comparison: GA vs Gradient Descent vs RL

When should you use each approach?

In [ ]:
comparison = """
| Aspect              | Genetic Algorithm    | Gradient Descent     | RL (Policy Gradient)  |
|---------------------|---------------------|----------------------|----------------------|
| **Gradients needed**| No                  | Yes                  | Yes                  |
| **Parallelization** | Excellent           | Limited              | Good                 |
| **Local minima**    | Escapes well        | Can get stuck        | Can get stuck        |
| **Sample efficiency**| Low                | High                 | Medium               |
| **Sparse rewards**  | Handles well        | N/A                  | Struggles            |
| **Scalability**     | Moderate            | Excellent            | Good                 |
| **Best for**        | Non-differentiable, | Supervised learning, | Sequential decisions,|
|                     | multi-modal         | differentiable       | long-term rewards    |

Use GA/Neuroevolution when:
- Reward is sparse or deceptive
- Environment is non-differentiable
- You have lots of compute for parallel evaluation
- Problem is multi-modal (many local optima)
- You want to evolve architecture, not just weights

Use Gradient-based methods when:
- You have gradients (supervised learning)
- Sample efficiency matters
- Problem is high-dimensional
- You have limited compute
"""
print(comparison)

## 7. FAANG Interview Questions

### Q1: How does a genetic algorithm work? What are the main operators?

**Answer**:

GA is an optimization algorithm inspired by natural selection:

1. **Initialize**: Create random population of candidate solutions (genomes)
2. **Evaluate**: Calculate fitness of each individual
3. **Select**: Choose parents based on fitness (tournament, roulette wheel, etc.)
4. **Crossover**: Combine parent genomes to create offspring
5. **Mutate**: Add random variations to offspring
6. **Replace**: Form new generation, possibly keeping elites
7. **Repeat** until convergence

**Key operators**:
- Selection: Tournament, roulette wheel, rank-based
- Crossover: Single-point, two-point, uniform
- Mutation: Gaussian noise, bit flip, gene swap

---

### Q2: What is neuroevolution and how does it differ from backpropagation?

**Answer**:

**Neuroevolution**: Evolving neural network weights (and/or architecture) using evolutionary algorithms instead of gradient descent.

| Aspect | Neuroevolution | Backpropagation |
|--------|---------------|----------------|
| Gradients | Not required | Required |
| Parallelization | Excellent | Limited |
| Credit assignment | Population-level | Per-sample |
| Architecture | Can evolve | Fixed |
| Sample efficiency | Low | High |
| Non-differentiable | Handles well | Cannot handle |

**Use neuroevolution when**: Reward is sparse/non-differentiable, want to evolve architecture, or have parallel compute.

---

### Q3: What is NEAT and why is speciation important?

**Answer**:

**NEAT** (NeuroEvolution of Augmenting Topologies) evolves both weights AND network topology.

**Key innovations**:
1. **Historical markings**: Track when genes were created for alignment during crossover
2. **Speciation**: Group similar networks together
3. **Complexification**: Start simple, add complexity only when beneficial

**Why speciation matters**:
- New innovations (added nodes/connections) initially hurt fitness
- Without protection, innovations die before they can optimize
- Speciation lets innovations compete within their niche, giving them time to improve

---

### Q4: Explain the exploration-exploitation tradeoff in GA.

**Answer**:

**Exploration**: Searching new areas of the solution space
**Exploitation**: Refining known good solutions

**GA levers**:
| Parameter | High value = More... |
|-----------|---------------------|
| Mutation rate | Exploration |
| Mutation strength | Exploration |
| Population size | Exploration |
| Selection pressure | Exploitation |
| Elite ratio | Exploitation |

**Balancing strategies**:
- Start with high exploration, reduce over time
- Adaptive mutation (increase when stuck)
- Diversity maintenance (niching, crowding)

---

### Q5: What are Evolution Strategies and how do they differ from GA?

**Answer**:

**Evolution Strategies (ES)**:
- Sample population: theta + sigma * noise
- Evaluate fitness
- Update theta toward high-fitness samples
- No crossover, just mutation and selection

**Key differences from GA**:
| Aspect | GA | ES |
|--------|----|----|  
| Crossover | Yes | No |
| Representation | Discrete genomes | Continuous distribution |
| Update | Replace population | Gradient-like update |
| Parallelization | Good | Excellent |

**Modern ES (like OpenAI ES)**: Can match RL performance on some tasks with better parallelization.

## 8. Key Takeaways

1. **Genetic Algorithms** optimize without gradients using selection, crossover, and mutation
2. **Neuroevolution** applies GAs to neural network weights (and potentially architecture)
3. **Selection strategies** (tournament, roulette, rank) control exploration-exploitation tradeoff
4. **NEAT** evolves topology, using speciation to protect innovation
5. **Evolution Strategies** are simpler (no crossover) and highly parallelizable
6. **Use GAs/ES when**: gradients unavailable, rewards sparse, architecture matters
7. **Elitism** preserves best solutions, mutation introduces variation